In [1]:
from pyspark.sql import SparkSession

import pyspark.sql.functions as F
import datetime
import os

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
source_path = 'datalake/raw'
file_name = 'products-1-1-'
target_path = 'datalake/cleaned'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df_raw_products = (
    spark
    .read
    .format("csv")
    .option("header",True)
    .load(f"./jupyter_workspace/{source_path}/raw_{file_name}.csv")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df_raw_products.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ProductID: string (nullable = true)
 |-- ProductDesc: string (nullable = true)
 |-- ProductNumber: string (nullable = true)
 |-- MakeFlag: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- SafetyStockLevel: string (nullable = true)
 |-- ReorderPoint: string (nullable = true)
 |-- StandardCost: string (nullable = true)
 |-- ListPrice: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- SizeUnitMeasureCode: string (nullable = true)
 |-- Weight: string (nullable = true)
 |-- WeightUnitMeasureCode: string (nullable = true)
 |-- ProductCategoryName: string (nullable = true)
 |-- ProductSubCategoryName: string (nullable = true)

In [5]:
df_raw_products.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---------------------------+-------------+--------+-----+----------------+------------+------------+---------+----+-------------------+------+---------------------+-------------------+----------------------+
|ProductID|ProductDesc                |ProductNumber|MakeFlag|Color|SafetyStockLevel|ReorderPoint|StandardCost|ListPrice|Size|SizeUnitMeasureCode|Weight|WeightUnitMeasureCode|ProductCategoryName|ProductSubCategoryName|
+---------+---------------------------+-------------+--------+-----+----------------+------------+------------+---------+----+-------------------+------+---------------------+-------------------+----------------------+
|680      |HL Road Frame - Black, 58  |FR-R92B-58   |True    |Black|500             |375         |1059.31     |1431.5   |58  |CM                 |2.24  |LB                   |null               |Road Frames           |
|706      |HL Road Frame - Red, 58    |FR-R92R-58   |True    |Red  |500             |375         |1059.31     |1431.5   |58 

In [9]:
df_store_products = (
    df_raw_products
    .select(
        df_raw_products.ProductID.cast('int').alias('pk_ProductID'),
        df_raw_products.ProductDesc,
        df_raw_products.ProductNumber,
        df_raw_products.MakeFlag.cast('boolean'),
        F.when(df_raw_products.Color.isNull(), F.lit('N/A')).otherwise(df_raw_products.Color).alias('Color'),
        df_raw_products.SafetyStockLevel.cast('int'),
        df_raw_products.ReorderPoint.cast('int'),
        df_raw_products.StandardCost.cast('float'),
        df_raw_products.ListPrice.cast('float'),
        df_raw_products.Size,
        df_raw_products.SizeUnitMeasureCode,
        df_raw_products.Weight.cast('float'),
        df_raw_products.WeightUnitMeasureCode,        
        df_raw_products.ProductCategoryName,        
        df_raw_products.ProductSubCategoryName,
        F.current_timestamp().alias('loadDateTimeJob')
    )    
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df_store_products.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- pk_ProductID: integer (nullable = true)
 |-- ProductDesc: string (nullable = true)
 |-- ProductNumber: string (nullable = true)
 |-- MakeFlag: boolean (nullable = true)
 |-- Color: string (nullable = true)
 |-- SafetyStockLevel: integer (nullable = true)
 |-- ReorderPoint: integer (nullable = true)
 |-- StandardCost: float (nullable = true)
 |-- ListPrice: float (nullable = true)
 |-- Size: string (nullable = true)
 |-- SizeUnitMeasureCode: string (nullable = true)
 |-- Weight: float (nullable = true)
 |-- WeightUnitMeasureCode: string (nullable = true)
 |-- ProductCategoryName: string (nullable = true)
 |-- ProductSubCategoryName: string (nullable = true)
 |-- loadDateTimeJob: timestamp (nullable = false)

In [12]:
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
(
    df_store_products
    .coalesce(1)
    .write
    .mode('overwrite')
    .parquet(f"./jupyter_workspace/{target_path}/store_{file_name}/")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…